In [2]:
# ATTENTION: Please do not alter any of the provided code in the exercise. Only add your own code where indicated
# ATTENTION: Please do not add or remove any cells in the exercise. The grader will check specific cells based on the cell position.
# ATTENTION: Please use the provided epoch values when training.

import csv
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from os import getcwd

In [2]:
def get_data(filename):
  # You will need to write code that will read the file passed
  # into this function. The first line contains the column headers
  # so you should ignore it
  # Each successive line contians 785 comma separated values between 0 and 255
  # The first value is the label
  # The rest are the pixel values for that picture
  # The function will return 2 np.array types. One with all the labels
  # One with all the images
  #
  # Tips: 
  # If you read a full line (as 'row') then row[0] has the label
  # and row[1:785] has the 784 pixel values
  # Take a look at np.array_split to turn the 784 pixels into 28x28
  # You are reading in strings, but need the values to be floats
  # Check out np.array().astype for a conversion
    with open(filename) as training_file:
        spamreader =csv.reader(training_file,delimiter=',')
        i=0
        x = list()
        label_1 = list()
        images_1 = list()
        for row in spamreader:    
            if i!=0:
                label_1.append(row[0])
                x = row[1:785]
                image = np.array_split(x,28)
                #if i==2: 
                    #print(images_1)
                images_1.append(image)
            i=i+1 
        images = np.array(images_1).astype('float')
        labels = np.array(label_1).astype('float')
        
        
      # Your code starts here
      # Your code ends here
    return images, labels

path_sign_mnist_train = f"{getcwd()}/../tmp2/sign_mnist_train.csv"
path_sign_mnist_test = f"{getcwd()}/../tmp2/sign_mnist_test.csv"
training_images, training_labels = get_data(path_sign_mnist_train)
testing_images, testing_labels = get_data(path_sign_mnist_test)

# Keep these
print(training_images.shape)
print(training_labels.shape)
print(testing_images.shape)
print(testing_labels.shape)

# Their output should be:
# (27455, 28, 28)
# (27455,)
# (7172, 28, 28)
# (7172,)

(27455, 28, 28)
(27455,)
(7172, 28, 28)
(7172,)


In [3]:
# In this section you will have to add another dimension to the data
# So, for example, if your array is (10000, 28, 28)
# You will need to make it (10000, 28, 28, 1)
# Hint: np.expand_dims

training_images = np.expand_dims(training_images,axis=3)# Your Code Here
testing_images = np.expand_dims(testing_images,axis=3) # Your Code Here

# Create an ImageDataGenerator and do Image Augmentation
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
    )

validation_datagen = ImageDataGenerator(rescale=1/255)
    
# Keep These
print(training_images.shape)
print(testing_images.shape)
    
# Their output should be:
# (27455, 28, 28, 1)
# (7172, 28, 28, 1)

(27455, 28, 28, 1)
(7172, 28, 28, 1)


In [4]:
# Define the model
# Use no more than 2 Conv2D and 2 MaxPooling2D
model = tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(64,(3,3),activation='relu',input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(26,activation='softmax')]
    )

# Compile Model. 
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the Model
history = model.fit_generator(train_datagen.flow(training_images,training_labels,batch_size=10),
                              steps_per_epoch=len(training_images)/10,
                              epochs=10,
                              validation_data=validation_datagen.flow(testing_images,testing_labels,batch_size=10))

model.evaluate(testing_images, testing_labels, verbose=0)

Epoch 1/15
2746/2745 [==============================] - 154s 56ms/step - loss: 2.6745 - accuracy: 0.1776 - val_loss: 1.8827 - val_accuracy: 0.3939
Epoch 2/15
2746/2745 [==============================] - 152s 56ms/step - loss: 1.8927 - accuracy: 0.3912 - val_loss: 1.2171 - val_accuracy: 0.5626
Epoch 3/15
2746/2745 [==============================] - 153s 56ms/step - loss: 1.5559 - accuracy: 0.4927 - val_loss: 1.1308 - val_accuracy: 0.6181
Epoch 4/15
2746/2745 [==============================] - 148s 54ms/step - loss: 1.3570 - accuracy: 0.5511 - val_loss: 0.8754 - val_accuracy: 0.6729
Epoch 5/15
2746/2745 [==============================] - 159s 58ms/step - loss: 1.2265 - accuracy: 0.5940 - val_loss: 0.7737 - val_accuracy: 0.7108
Epoch 6/15
2746/2745 [==============================] - 152s 55ms/step - loss: 1.1220 - accuracy: 0.6279 - val_loss: 0.8440 - val_accuracy: 0.7055
Epoch 7/15
2746/2745 [==============================] - 154s 56ms/step - loss: 1.0348 - accuracy: 0.6557 - val_loss: 0

[564.1358702830343, 0.4090909]

In [5]:
# Plot the chart for accuracy and loss on both training and validation
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

NameError: name 'accuracy' is not defined

# Submission Instructions

In [ ]:
# Now click the 'Submit Assignment' button above.

# When you're done or would like to take a break, please run the two cells below to save your work and close the Notebook. This will free up resources for your fellow learners. 

In [ ]:
%%javascript
<!-- Save the notebook -->
IPython.notebook.save_checkpoint();